In [3]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np
#run jupyter viewer
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("for_dash.csv")
spacex_df.info()
df1 = spacex_df[spacex_df.isna().any(axis=1)]
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       120 non-null    int64 
 1   Flight No.       120 non-null    int64 
 2   Launch site      120 non-null    object
 3   Payload mass     120 non-null    int64 
 4   Orbit            120 non-null    object
 5   Customer         120 non-null    object
 6   Version Booster  115 non-null    object
 7   Date             120 non-null    object
 8   Time             120 non-null    object
 9   Class            120 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 9.5+ KB


,Unnamed: 0,Flight No.,Launch site,Payload mass,Orbit,Customer,Version Booster,Date,Time,Class
6,6,7,CCAFS,3170,GTO,SES,NaN,3 December 2013,22:41,0
7,7,8,CCAFS,3325,GTO,Thaicom,NaN,6 January 2014,22:06,0
8,8,9,Cape Canaveral,2296,LEO,NASA (CRS),NaN,18 April 2014,19:25,0
9,9,10,Cape Canaveral,1316,LEO,Orbcomm,NaN,14 July 2014,15:15,0
10,10,11,Cape Canaveral,4535,GTO,AsiaSat,NaN,5 August 2014,08:00,0


In [4]:
spacex_df['Version Booster'] =  spacex_df['Version Booster'].replace(np.nan, 'F9 v1.1')

In [5]:
df1 = spacex_df[spacex_df.isna().any(axis=1)]
df1

,Unnamed: 0,Flight No.,Launch site,Payload mass,Orbit,Customer,Version Booster,Date,Time,Class


In [8]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output

pd.options.plotting.backend = "plotly"


#run jupyter viewer
# Read the airline data into pandas dataframe
max_payload = spacex_df['Payload mass'].max()
min_payload = spacex_df['Payload mass'].min()


pd.options.plotting.backend = "plotly"
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout

sites=spacex_df['Launch site'].unique().tolist()
sites.insert(0,'All Sites')




app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                   id='site-dropdown',
                                   options=[
                                    {'label': i, 'value': i} for i in sites],
                                    placeholder="Select a Launch Site here",
                                    value='All Sites',
                                    searchable=True),
                            
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0,
                                max=10000,
                                step=1000,
                                value=[min_payload,max_payload],
                                marks={
                                0: '0 kg',
                                2500: '2500',
                                5000: '5000',
                                7500: '7500',
                                10000: '10000'
                                }),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
Output(component_id='success-pie-chart', component_property='figure'),
Input(component_id='site-dropdown', component_property='value')
)
def get_pie(value):
    filtered_df = spacex_df
    if value == 'All Sites':
        fig = px.pie(filtered_df, values='Class', names='Launch site', title='Total Success Launches By Site')
        return fig

    else:
        filtered_df = spacex_df[spacex_df['Launch site'] == value].groupby(['Launch site', 'Class']). \
        size().reset_index(name='Class count')
        title = f"Total Success Launches for site {value}"
        fig = px.pie(filtered_df,values='Class count', names='Class', title=title)
        return fig

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
Output(component_id='success-payload-scatter-chart', component_property='figure'),
[Input(component_id='site-dropdown', component_property='value'),
Input(component_id='payload-slider', component_property='value')]
)

def get_scatter(value1,value2):
    filtered_df2_1=spacex_df[(spacex_df['Payload mass'] > value2[0]) & (spacex_df['Payload mass'] < value2[1])]

    if value1=='All Sites':
        fig= px.scatter(filtered_df2_1,x="Payload mass",y="Class",color="Version Booster",\
        title="Correlation between Payload and Success for All sites")
        return fig
    else :
        filtered_df2_2=filtered_df2_1[filtered_df2_1['Launch site']==value1]
        fig= px.scatter(filtered_df2_2,x="Payload mass",y="Class",color="Version Booster",\
        title=f"Correlation between Payload and Success for site {value1}")
        return fig
# Run the app
app.run_server(mode='inline', port = 8080, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)
